In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [2]:
spark

In [3]:
df=spark.read.format("csv").option("header","true").option("inferSchema","true").load("us_census.csv")

In [19]:
#dff = spark.read.csv("us_census.csv", header=True, inferSchema=True)

In [4]:
df

DataFrame[State FIPS: int, State: string, 116th Congressional District: int, 2017 NAICS Code: string, NAICS Description: string, Number of Establishments: string, Employment: string, Employment Noise Flag: string, 1st Quarter Payroll ($1,000): string, 1st Quarter Payroll Noise Flag: string, Annual Payroll ($1,000): string, Annual Payroll Noise Flag: string]

In [10]:
df.show()

+----------+-------+----------------------------+---------------+--------------------+------------------------+----------+---------------------+----------------------------+------------------------------+-----------------------+-------------------------+
|State FIPS|  State|116th Congressional District|2017 NAICS Code|   NAICS Description|Number of Establishments|Employment|Employment Noise Flag|1st Quarter Payroll ($1,000)|1st Quarter Payroll Noise Flag|Annual Payroll ($1,000)|Annual Payroll Noise Flag|
+----------+-------+----------------------------+---------------+--------------------+------------------------+----------+---------------------+----------------------------+------------------------------+-----------------------+-------------------------+
|         1|Alabama|                           1|         ------|Total for all sec...|                  15,921|  2,44,615|                    G|                   24,92,282|                             G|            1,04,17,508|       

In [27]:
#data types of columns
df.dtypes

[('State FIPS', 'int'),
 ('State', 'string'),
 ('116th Congressional District', 'int'),
 ('2017 NAICS Code', 'string'),
 ('NAICS Description', 'string'),
 ('Number of Establishments', 'string'),
 ('Employment', 'string'),
 ('Employment Noise Flag', 'string'),
 ('1st Quarter Payroll ($1,000)', 'string'),
 ('1st Quarter Payroll Noise Flag', 'string'),
 ('Annual Payroll ($1,000)', 'string'),
 ('Annual Payroll Noise Flag', 'string')]

In [28]:
#rename collumns
df_rename= df.withColumnRenamed('State FIPS','State_FIPS').withColumnRenamed('116th Congressional District','116_Congressional District')

In [29]:
df_rename.columns

['State_FIPS',
 'State',
 '116_Congressional District',
 '2017 NAICS Code',
 'NAICS Description',
 'Number of Establishments',
 'Employment',
 'Employment Noise Flag',
 '1st Quarter Payroll ($1,000)',
 '1st Quarter Payroll Noise Flag',
 'Annual Payroll ($1,000)',
 'Annual Payroll Noise Flag']

In [8]:
#rename method-2
from pyspark.sql import functions as F

renamed_df = df.select([F.col(col).alias(col.replace(' ', '_')) for col in df.columns])

In [9]:
renamed_df.columns

['State_FIPS',
 'State',
 '116th_Congressional_District',
 '2017_NAICS_Code',
 'NAICS_Description',
 'Number_of_Establishments',
 'Employment',
 'Employment_Noise_Flag',
 '1st_Quarter_Payroll_($1,000)',
 '1st_Quarter_Payroll_Noise_Flag',
 'Annual_Payroll_($1,000)',
 'Annual_Payroll_Noise_Flag']

In [47]:
x=df.describe('State FIPS','116th Congressional District')
x.show()

+-------+------------------+----------------------------+
|summary|        State FIPS|116th Congressional District|
+-------+------------------+----------------------------+
|  count|              9104|                        9104|
|   mean|27.715619507908613|          10.103910369068542|
| stddev|16.247660117346232|           10.64842522491003|
|    min|                 1|                           0|
|    max|                72|                          53|
+-------+------------------+----------------------------+



In [75]:
from pyspark.sql.functions import col
df_diff = df.withColumn('pay_roll_diff', df['Annual Payroll ($1,000)'] - df['1st Quarter Payroll ($1,000)'])
# df_diff.select("pay_roll_diff").show()
df_diff.where(col("pay_roll_diff").isNotNull()).select("pay_roll_diff").show()

+-------------+
|pay_roll_diff|
+-------------+
|        403.0|
|        225.0|
|        348.0|
|        912.0|
|        191.0|
|        427.0|
|        472.0|
|        601.0|
|        367.0|
|        402.0|
|        285.0|
|        557.0|
|        452.0|
|        253.0|
|        281.0|
|        337.0|
|        687.0|
|        390.0|
|        545.0|
|        729.0|
+-------------+
only showing top 20 rows



In [76]:
df_diff=df_diff.groupby("State")

In [77]:
df_avg=df_diff.avg("pay_roll_diff")
df_avg.show()

+--------------------+------------------+
|               State|avg(pay_roll_diff)|
+--------------------+------------------+
|                Utah|             511.0|
|              Hawaii|             476.0|
|           Minnesota|            452.75|
|                Ohio|             330.9|
|            Arkansas|            423.75|
|              Oregon|             436.0|
|               Texas| 482.2972972972973|
|        North Dakota|              null|
|        Pennsylvania| 355.3157894736842|
|         Connecticut|            415.75|
|            Nebraska| 548.6666666666666|
|             Vermont|              null|
|              Nevada|             365.5|
|         Puerto Rico|             546.0|
|          Washington|             462.0|
|            Illinois| 286.6818181818182|
|            Oklahoma|            413.25|
|District of Columbia|              null|
|            Delaware|              null|
|              Alaska|              null|
+--------------------+------------

In [19]:
renamed_df=renamed_df.withColumnRenamed('Annual_Payroll_($1,000)','Annual_Payroll')
renamed_df=renamed_df.select([eval('renamed_df.' + x + '.isNull().cast("int").alias("' + x + '")') for x in renamed_df.columns]).show(n=20)

+----------+-----+----------------------+----------+-----------------+------------------------+----------+---------------------+---------------+--------------------------+--------------+-------------------------+
|State_FIPS|State|Congressional_District|NAICS_Code|NAICS_Description|Number_of_Establishments|Employment|Employment_Noise_Flag|Quarter_Payroll|Quarter_Payroll_Noise_Flag|Annual_Payroll|Annual_Payroll_Noise_Flag|
+----------+-----+----------------------+----------+-----------------+------------------------+----------+---------------------+---------------+--------------------------+--------------+-------------------------+
|         0|    0|                     0|         0|                0|                       0|         0|                    0|              0|                         0|             0|                        0|
|         0|    0|                     0|         0|                0|                       0|         0|                    0|              0|    

In [4]:
from pyspark import SQLContext

sqlContext = SQLContext(sc)

In [6]:
from pyspark.sql import Row
l = [('Ankit',25),('Jalfaizy',22),('saurabh',20),('Bala',26)]
rdd = sc.parallelize(l)
people = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
schemaPeople = sqlContext.createDataFrame(people)

In [22]:
type(schemaPeople)

pyspark.sql.dataframe.DataFrame

In [23]:
schemaPeople.show()

+--------+---+
|    name|age|
+--------+---+
|   Ankit| 25|
|Jalfaizy| 22|
| saurabh| 20|
|    Bala| 26|
+--------+---+



In [17]:
schemaPeople.describe().show()

+-------+-------+------------------+
|summary|   name|               age|
+-------+-------+------------------+
|  count|      4|                 4|
|   mean|   null|             23.25|
| stddev|   null|2.7537852736430515|
|    min|  Ankit|                20|
|    max|saurabh|                26|
+-------+-------+------------------+



In [20]:
schemaPeople.registerTempTable('schemaPeople')

In [21]:
sqlContext.sql('select name from schemaPeople').show(5)

+--------+
|    name|
+--------+
|   Ankit|
|Jalfaizy|
| saurabh|
|    Bala|
+--------+

